In [5]:
import time

def benchmark_sage(n, k, iterations):
    times = []
    for _ in range(iterations):
        start_time = time.time()
        binomial(n, k)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = sum(times) / iterations
    print(f"SageMath: Tempo médio para binomial({n}, {k}) em {iterations} iterações: {avg_time:.6f} segundos")

# Parâmetros do benchmark
n = 13456
k = 345
iterations = 1000

benchmark_sage(n, k, iterations)

SageMath: Tempo médio para binomial(13456, 345) em 1000 iterações: 0.000045 segundos


In [6]:
import time
from scipy.special import comb

def benchmark_python(n, k, iterations):
    times = []
    for _ in range(iterations):
        start_time = time.time()
        comb(n, k, exact=True)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = sum(times) / iterations
    print(f"Python: Tempo médio para comb({n}, {k}) em {iterations} iterações: {avg_time:.6f} segundos")

# Parâmetros do benchmark
n = 13456
k = 345
iterations = 1000

benchmark_python(n, k, iterations)

Python: Tempo médio para comb(13456, 345) em 1000 iterações: 0.000112 segundos


In [4]:
load("util.sage")
def coeff_de_moebius(poly, graus, graus_total, chaves, extremos, novo_coefsxy):    
    soma_total = 0

    binomials_cache = {}
    powers_cache = {}

    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]

            somak = 0
            for pp in range(ck + 1):
                binom_ck_pp = binomials_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binomials_cache[(ck, pp)] = binom_ck_pp

                binom_gtck_gkspp = binomials_cache.get((gt - ck, gks - pp))
                if binom_gtck_gkspp is None:
                    binom_gtck_gkspp = binomial(gt - ck, gks - pp)
                    binomials_cache[(gt - ck, gks - pp)] = binom_gtck_gkspp

                a_k_pp = powers_cache.get((a_k, pp))
                if a_k_pp is None:
                    a_k_pp = pow(a_k, pp)
                    powers_cache[(a_k, pp)] = a_k_pp

                b_k_ck_pp = powers_cache.get((b_k, ck - pp))
                if b_k_ck_pp is None:
                    b_k_ck_pp = pow(b_k,(ck - pp))
                    powers_cache[(b_k, ck - pp)] = b_k_ck_pp

                somak += binom_ck_pp * binom_gtck_gkspp * a_k_pp * b_k_ck_pp

            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total += reduce(lambda a, b: a * b, somas_e_valor)

    return soma_total

In [5]:
sage: u, v = var('u v')
sage: poly_test = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4
sage: degrees_test = [1, 2]  # To find the coefficient of (x_1)^1 * (x_2)^2
sage: bounds_test = [(3, 5), (7, 11)]
sage: new_coeffs_xy_test = polynomial_to_dict(poly_test)  # Must be a global variable
sage: c = coeff_de_moebius(poly_test, degrees_test, bounds_test, new_coeffs_xy_test)
sage: c

TypeError: coeff_de_moebius() missing 2 required positional arguments: 'extremos' and 'novo_coefsxy'

In [4]:
sage: u, v = var('u v')
sage: p3 = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4
sage: polynomial_to_dict(p3)

{(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}

In [7]:
var('u, x, v, y, z')
poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
variaveis = poly.free_variables()
extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
novo_coefsxy_teste = polynomial_to_dict(poly)

In [9]:
moebius_coefficients(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste)

43188488958

In [12]:
def coeficientes_de_moebius_simbolico_varias_variaveis(poly, graus, extremos, novo_coefsxy ):
    """INPUTS -
    poly- Um polinomio como expressão simbolica na classe  <class 'sage.symbolic.expression.Expression'>;
    graus - Uma lista com naturais representando o multigrau do monomio que voce quer o coeficiente.
            Importante - A sua lista de graus deve estar na ordem que o Sage ordena as variaveis.
    extremos - Uma lista com pares (a_i,b_i) que dão o intervalo da substituição de Moebius para
    a variavel x_i, isto é,    x_i---> x_i = (a_i*u_i+b_i)/(u_i+1) para a variavel u_i.
    OUTPUT - O coeficiente do monomio com multigrau dado pela lista graus depois da substituicao de Moebius
    Exemplo - 1
    sage: poly_teste = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4;
    sage: graus_teste = [1,2]; ## para acahar o coeficiente de (x_1)^1*(x_2)^2
    sage: extremos_teste = [(3, 5), (7, 11)];
    sage: novo_coefsxy_teste = dicionario_geral(poly_teste); # virar variável global;
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly_teste, graus_teste, extremos_teste, novo_coefsxy_teste)
        1133944
    Exemplo - 2
    sage: poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = poly.free_variables(); variaveis
    sage: extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
    sage: ru, rx, ry, rz = var('ru, rx, ry, rz')
    sage: variaveis_r = [ru, rx, ry, rz]
    sage: novo_coefsxy_teste_2 = dicionario_geral(poly); 
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste_2) ## Procurando o coeficiente de ru^0 * rx^0 * ry^1 * rz^0
    sage: show(c)
     244374065874
       
       OBSERVACAO: Para mais detalhes veja o arquivo /Artigo_de_Aplicacoes/Reuniao-casa-marcelo-26-07/Icosaedro-bloco-2-reunião-casa-marcelo/Funcao_Para_Coeficientes_Com_Varias_Variaveis_Versao_Final.ipynb
        """
    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
    graus_total = [poly.degree(x) for x in variaveis];
    chaves = novo_coefsxy.keys()
    soma_total = 0
    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            somak = 0
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]
            for pp in range(ck+1):
                somak = somak+((binomial(ck,pp))*(binomial(gt-ck, gks-pp))*((a_k)^(pp))*((b_k)^(ck-pp)))
            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total = soma_total + reduce(lambda a, b: a*b, somas_e_valor)
    return soma_total

from functools import reduce
from sage.all import binomial

def dicionario_geral_gpt(poly):
    def nova_lista(lista, var):
        new_lista = []
        for item in lista:
            sublista = item[0].coefficients(var)
            for coef, monom in sublista:
                new_lista.append([coef, monom + item[-1]])
        return new_lista

    variaveis = poly.variables()
    variavel_1 = variaveis[0]
    l_variavel_1 = poly.coefficients(variavel_1)
    l = [[item[0], item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(reversed(j[1])): j[0] for j in l}
    return dicionario

def coeficientes_de_moebius_simbolico_varias_variaveis_gpt(poly, graus, extremos, novo_coefsxy):
    variaveis = poly.variables()
    graus_total = [poly.degree(x) for x in variaveis]
    chaves = novo_coefsxy.keys()
    soma_total = 0

    binomials_cache = {}
    powers_cache = {}

    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]

            somak = 0
            for pp in range(ck + 1):
                binom_ck_pp = binomials_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binomials_cache[(ck, pp)] = binom_ck_pp

                binom_gtck_gkspp = binomials_cache.get((gt - ck, gks - pp))
                if binom_gtck_gkspp is None:
                    binom_gtck_gkspp = binomial(gt - ck, gks - pp)
                    binomials_cache[(gt - ck, gks - pp)] = binom_gtck_gkspp

                a_k_pp = powers_cache.get((a_k, pp))
                if a_k_pp is None:
                    a_k_pp = a_k^pp
                    powers_cache[(a_k, pp)] = a_k_pp

                b_k_ck_pp = powers_cache.get((b_k, ck - pp))
                if b_k_ck_pp is None:
                    b_k_ck_pp = b_k^(ck - pp)
                    powers_cache[(b_k, ck - pp)] = b_k_ck_pp

                somak += binom_ck_pp * binom_gtck_gkspp * a_k_pp * b_k_ck_pp

            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total += reduce(lambda a, b: a * b, somas_e_valor)

    return soma_total

from functools import reduce

def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario



In [13]:
sage: poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
sage: variaveis = poly.free_variables(); variaveis
sage: extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
sage: ru, rx, ry, rz = var('ru, rx, ry, rz')
sage: variaveis_r = [ru, rx, ry, rz]
sage: novo_coefsxy_teste_2 = dicionario_geral(poly); 
sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste_2) ## Procurando o coeficiente de ru^0 * rx^0 * ry^1 * rz^0
sage: show(c)

244374065874